# Travel Stack Exchange
**Data Processing in R and Python**

cutieskye

2022-01-12

---

## Introduction

For this project, I will analyze and manipulate the [Travel Stack Exchange](https://travel.stackexchange.com/) simplified anonymized data dump of user-contributed content in Python using the `pandas` library.

In [1]:
import os
import pandas as pd
import sqlite3
import tempfile

I will refer to the global system table to dynamically set variables. The justification is to leave the queries intact and avoid code repetition. Otherwise, creating a dictionary with names of variables as keys would have been a better approach.

In [2]:
data = ["Badges", "Comments", "PostLinks", "Posts", "Tags", "Users", "Votes"]
for datum in data:
    globals()[datum] = pd.read_csv(
        f"travel_stackexchange_com/{datum}.csv.gz", compression="gzip")

The following chunk sets up the database enabling us to read the SQL queries and examine the reference solutions.

In [3]:
db = os.path.join(tempfile.mkdtemp(), "example.db")
if os.path.isfile(db):
    os.remove(db)

I connect to the database and populate it with our data. Now I can create dataframes corresponding to the result sets of each SQL query. Note that I added *ref6* on my own, which is simply the result of the fifth query without a `LIMIT` clause. Freeing unused resources is a good programming practice, so I close the connection to the database in the end.

In [4]:
con = sqlite3.connect(db)

for datum in data:
    globals()[datum].to_sql(f"{datum}", con, if_exists="replace")

ref1 = pd.read_sql_query(
    """
        SELECT
            Name,
            COUNT(*) AS Number,
            MIN(Class) AS BestClass
        FROM Badges
        GROUP BY Name
        ORDER BY Number DESC
        LIMIT 10
    """, con)

ref2 = pd.read_sql_query(
    """
        SELECT Location, COUNT(*) AS Count
        FROM (
            SELECT Posts.OwnerUserId, Users.Id, Users.Location
            FROM Users
            JOIN Posts ON Users.Id = Posts.OwnerUserId
        )
        WHERE Location NOT IN ('')
        GROUP BY Location
        ORDER BY Count DESC
        LIMIT 10
    """, con)

ref3 = pd.read_sql_query(
    """
    SELECT
        Users.AccountId,
        Users.DisplayName,
        Users.Location,
        AVG(PostAuth.AnswersCount) as AverageAnswersCount
      FROM
      (
        SELECT
          AnsCount.AnswersCount,
          Posts.Id,
          Posts.OwnerUserId
        FROM (
          SELECT Posts.ParentId, COUNT(*) AS AnswersCount
          FROM Posts
          WHERE Posts.PostTypeId = 2
          GROUP BY Posts.ParentId
        ) AS AnsCount
        JOIN Posts ON Posts.Id = AnsCount.ParentId
      ) AS PostAuth
      JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
      GROUP BY OwnerUserId
      ORDER BY AverageAnswersCount DESC
      LIMIT 10
      """, con)

ref4 = pd.read_sql_query(
    """
        SELECT
            Posts.Title,
            UpVotesPerYear.Year,
            MAX(UpVotesPerYear.Count) AS Count
        FROM (
            SELECT
                PostId,
                COUNT(*) AS Count,
                STRFTIME('%Y', Votes.CreationDate) AS Year
            FROM Votes
            WHERE VoteTypeId=2
            GROUP BY PostId, Year
        ) AS UpVotesPerYear
        JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
        WHERE Posts.PostTypeId=1
        GROUP BY Year
        ORDER BY Year ASC
    """, con)

ref5 = pd.read_sql_query(
    """
        SELECT
          Posts.Title,
          VotesByAge2.OldVotes
        FROM Posts
        JOIN (
          SELECT
            PostId,
            MAX(CASE WHEN VoteDate = 'new'THEN Total ELSE 0 END) NewVotes,
            MAX(CASE WHEN VoteDate = 'old'THEN Total ELSE 0 END) OldVotes,
            SUM(Total) AS Votes
          FROM (
            SELECT
              PostId,
              CASE STRFTIME('%Y', CreationDate)
                WHEN '2021'THEN 'new'
                WHEN '2020'THEN 'new'
                ELSE 'old'
                END VoteDate,
              COUNT(*) AS Total
            FROM Votes
            WHERE VoteTypeId IN (1, 2, 5)
            GROUP BY PostId, VoteDate
          ) AS VotesByAge
          GROUP BY VotesByAge.PostId
          HAVING NewVotes=0
        ) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
        WHERE Posts.PostTypeId=1
        ORDER BY VotesByAge2.OldVotes DESC
        LIMIT 10
    """, con)

ref6 = pd.read_sql_query(
    """
        SELECT
          Posts.Title,
          VotesByAge2.OldVotes
        FROM Posts
        JOIN (
          SELECT
            PostId,
            MAX(CASE WHEN VoteDate = 'new'THEN Total ELSE 0 END) NewVotes,
            MAX(CASE WHEN VoteDate = 'old'THEN Total ELSE 0 END) OldVotes,
            SUM(Total) AS Votes
          FROM (
            SELECT
              PostId,
              CASE STRFTIME('%Y', CreationDate)
                WHEN '2021'THEN 'new'
                WHEN '2020'THEN 'new'
                ELSE 'old'
                END VoteDate,
              COUNT(*) AS Total
            FROM Votes
            WHERE VoteTypeId IN (1, 2, 5)
            GROUP BY PostId, VoteDate
          ) AS VotesByAge
          GROUP BY VotesByAge.PostId
          HAVING NewVotes=0
        ) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
        WHERE Posts.PostTypeId=1
        ORDER BY VotesByAge2.OldVotes DESC
    """, con)

con.close()

This function compares dataframes in pairs, allowing us to pass any number of them as input. Dataframes are considered equivalent irrespectively of their row order. This is achieved by sorting every dataframe by all column names.

In [5]:
def are_equivalent(dataframes):
    sorted_dataframes = []
    for df in dataframes:
        sorted_dataframes.append(df.sort_values(
            by=df.columns.to_list()).reset_index(drop=True))
    return [df1.equals(df2) for df1, df2 in zip(sorted_dataframes, sorted_dataframes[1:])]

## Query 1

In [6]:
res1 = Badges[["Name", "Class"]].groupby("Name", as_index=False).agg(Number=("Name", "size"), BestClass=(
    "Class", "min")).sort_values(by="Number", ascending=False, ignore_index=True)[:10]

res1

,Name,Number,BestClass
0,Autobiographer,24564,3
1,Student,23930,3
2,Supporter,17826,3
3,Popular Question,14840,3
4,Nice Answer,12406,3
5,Yearling,11281,2
6,Teacher,9746,3
7,Editor,9115,3
8,Notable Question,8613,2
9,Scholar,7224,3


In [7]:
are_equivalent([ref1, res1])[0]

True

## Query 2

In [8]:
res2 = Users[Users["Location"] != ""][["Id", "Location"]].merge(Posts["OwnerUserId"], left_on="Id", right_on="OwnerUserId").groupby(
    "Location").size().to_frame("Count").reset_index().sort_values(by="Count", ascending=False, ignore_index=True)[:10]
    
res2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [9]:
are_equivalent([ref2, res2])[0]

True

## Query 3

The third query turned out to be tricky to be reproduced step by step, primarily due to subtle differences in how `SQLite` and `pandas` handle missing data. Fortunately, the final result is not influenced by this.

In [10]:
ans_count = Posts[Posts["PostTypeId"] == 2].groupby(
    "ParentId").size().to_frame("AnswersCount")

post_auth = ans_count.merge(Posts[["Id", "OwnerUserId"]], left_on="ParentId", right_on="Id")[
    ["AnswersCount", "OwnerUserId"]].groupby("OwnerUserId").agg(AverageAnswersCount=("AnswersCount", "mean"))
    
users_info = Users[["AccountId", "DisplayName",
                    "Location"]].set_index("AccountId")
                    
res3 = post_auth.join(users_info, how="inner").reset_index().rename(columns={"index": "AccountId"}).reset_index().sort_values(
    "AverageAnswersCount", ascending=False)[["AccountId", "DisplayName", "Location", "AverageAnswersCount"]][:10]

res3

,AccountId,DisplayName,Location,AverageAnswersCount
472,40811.0,vocaro,"San Jose, CA",11.0
14,280.0,csmba,"San Francisco, CA",11.0
11,204.0,Josh,Australia,10.0
511,44093.0,Emma Arbogast,"Salem, OR",10.0
181,11758.0,rvarcher,"Oklahoma City, OK",9.0
764,79346.0,Thomas Matthews,California,8.0
598,54571.0,Christian,NaN,8.0
239,19588.0,JD Isaacks,"Atlanta, GA",8.0
490,42364.0,Petrogad,NaN,8.0
254,20473.0,Jeremy Boyd,"Houston, TX",8.0


In [11]:
are_equivalent([ref3, res3])[0]

True

## Query 4

Bare columns are the biggest challenge in translating these queries. If the original query has an aggregate function `MAX`, bare columns must take values from the row which contains the maximum. For this purpose there exists `idxmax()`.

In [12]:
up_votes_per_year = Votes[Votes["VoteTypeId"] == 2][[
    "PostId", "CreationDate"]].rename(columns={"CreationDate": "Year"})

up_votes_per_year["Year"] = up_votes_per_year["Year"].str[:4]

up_votes_per_year = up_votes_per_year.groupby(
    ["PostId", "Year"], as_index=False).agg(Count=("PostId", "count"))

res4 = up_votes_per_year.merge(
    Posts[Posts.PostTypeId == 1][["Id", "Title"]], left_on="PostId", right_on="Id")
    
res4 = res4.loc[res4.groupby(
    "Year")["Count"].idxmax()][["Title", "Year", "Count"]].reset_index(drop=True)

res4

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


In [13]:
are_equivalent([ref4, res4])[0]

True

## Query 5

Here, we need to drop *PostId*'s for which new votes exist. We will not use them for grouping.

In [14]:
votes_by_age = Votes[Votes["VoteTypeId"].isin([1, 2, 5])]

new_votes = votes_by_age[votes_by_age["CreationDate"].str[:4]
                         >= "2020"]["PostId"]

votes_by_age = votes_by_age[~votes_by_age["PostId"].isin(new_votes)]

votes_by_age2 = votes_by_age.groupby("PostId", as_index=False).agg(
    OldVotes=("PostId", "count"))[["PostId", "OldVotes"]]

res5 = votes_by_age2.merge(Posts[Posts.PostTypeId == 1][["Id", "Title"]], left_on="PostId", right_on="Id")[
    ["Title", "OldVotes"]].sort_values(by="OldVotes", ascending=False, ignore_index=True)

res6 = res5.copy()

res5 = res5[:10]

res5

,Title,OldVotes
0,How to avoid drinking vodka?,200
1,"How can I do a ""broad"" search for flights?",175
2,How to convince airport security that I am not...,119
3,Can I accidentally miss the in-flight food?,114
4,How can black travellers deal with unwanted ha...,108
5,Why are there no seat belts on trains?,106
6,"How do you avoid ""tourist traps"" when travelin...",104
7,Can I fly with a gold bar?,102
8,Best way to dispose of a carry-on bag in Hong ...,99
9,How to spot fake reviews on TripAdvisor?,97


In [15]:
are_equivalent([ref5, res5])[0]

False

In [16]:
ref5

,Title,OldVotes
0,How to avoid drinking vodka?,200
1,"How can I do a ""broad"" search for flights?",175
2,How to convince airport security that I am not...,119
3,Can I accidentally miss the in-flight food?,114
4,How can black travellers deal with unwanted ha...,108
5,Why are there no seat belts on trains?,106
6,"How do you avoid ""tourist traps"" when travelin...",104
7,Can I fly with a gold bar?,102
8,Best way to dispose of a carry-on bag in Hong ...,99
9,How to successfully haggle / bargain in markets,97


Here, we have a mismatch in the last row. This is not a problem because multiple posts had 97 old votes. If we look at all records in these dataframes, we find that they are equal.

In [17]:
are_equivalent([ref6, res6])[0]

True

## Conclusion

Python's design philosophy (e.g., code readability) and capabilities of its third-party libraries like `numpy` or `pandas` have made it one of the most frequently used languages for scientific computing, data processing, and manipulation.